<h2> Files</h2>
<body>
    <b>smiles </b>: Virtual Environment - Use source /smiles/bin/activate to activate the virtual env<br>
    <b>datasets/download.sh </b>: Bash script to download datasets 
		Usage : bash download.sh gdb13 OR bash download.sh gdb17 or bash download.sh gdb17
<br>
    <b>datasets/delete.sh </b>: bash script to delete files in the folder. 
		Usage : bash delete.sh gdb11_ where gdb11 is the grep string to search for.
<br>
    <b>datasets/extract.sh </b>: To extract files from a tar file and to save it in a folder

<br>

 

# Functions

In [2]:
# Function to One hot encode the data into into 2 Dimensions. 
# Splitting the dataset to len(distinct_charachter) number of columns
def OneHotEncoding(data,ohe_dict,max):
  """ The function makes use of the ohe_helper function to One Hot Encode a list of charachters and to return 
  the encoded format. Since the input here is a string, the output will be of 2 Dimensions """

  import numpy as np
  ohe=[]
  for data_point in data:
    ohe.append(ohe_helper(data_point,ohe_dict,max))
  return ohe

def ohe_helper(data,ohe_dict,max_value):
  """ Helper function to One Hot Encode the data. 
  It is used by the above function"""

  import numpy as np
  out=np.zeros((max_value,len(ohe_dict)))
  for i,element in enumerate(data):
    out[i][key_value[element]]=1
  return out

def save(file,key_values):
  """ The function will take the data and save it as an npz file, with
  name smiles.npz"""
  import numpy as np
  np.savez('smiles.npz',ohe=temp,key=key_values)

def OneHotDecoding_helper(data,dictionary):
  """ Function to decode a One Hot Encoded 2D data back into its smile representation"""
  key=list(dictionary.keys())
  string=''
  for charachter in data:
    idx=charachter.argmax()
    string+=key[idx]
  return string

def check_smiles(string):
  object=check_molecule(string)

# Define auto encoder

In [3]:
def print_distributions(number_of_neurons, data):
  import matplotlib.pyplot as plt
  import seaborn as sns
  
  plt.figure(figsize=(20,20))

  for i in range(number_of_neurons):
    plt.subplot(8,number_of_neurons//8,i+1)
    sns.histplot(data[i])
    
def print_decoder_outputs(input,predictions,number):
  import matplotlib.pyplot as plt
  plt.figure(figsize=(10,3))
  for i in range(number):
    plt.subplot(1,number,i+1)
    plt.axis='off'
    plt.imshow(input[i].reshape(28,28),cmap='gray')
  plt.figure(figsize=(10,3))
  for i in range(number):
    plt.subplot(1,number,i+1)
    plt.axis='off'
    plt.imshow(predictions[i].reshape(28,28),cmap='gray')


# Preparing the data

In [4]:
# Connecting to google drive and navigating to folder
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks/molecule_generation_SMILES/scripts")

Mounted at /content/drive


In [5]:
!pip install smilite
# Importing Libraries
import pandas as pd
import smilite # Library to check if data generated is correct or not
from check_molecule import *

In [67]:
% time
data=pd.read_csv('gdb11_size10.smi',delimiter='\t',header=None)
# Renaming the columns
data.columns=['Molecule','_','__']

# Dropping the extra columns
data.drop(data.columns[-2:],axis=1,inplace=True)

# Fetching a smaller portion of the dataset
data=data.iloc[0:500000]

data.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


,Molecule
0,CC(C(C)(C)C)C(C)(C)C
1,CC(C)(C)C(N)C(C)(C)C
2,CC(C)(C)C(O)C(C)(C)C
3,CC(C)(C)C(F)C(C)(C)C
4,CC(C(C)(C)C)C(C)(C)N


<h4>One Hot Encoding the data

In [7]:
% time

# Finding the distinct charachters in the dataset
distinct_charachters=set(' '.join(data['Molecule'].values))

# Creating a dictionary to get the index value
key_value={}
key_value[' ']=0
idx=1
for charachter in distinct_charachters:
  if charachter!=' ':
    key_value[charachter]=idx
    idx+=1

# Creating a new column with the charachter seperated as a list
data['Molecule_sep']=data['Molecule'].apply(lambda x: list(x))

#Creating a new column with length of each smiles representation
data['length']=data['Molecule_sep'].apply(lambda x: len(x))
data=data[data['length']==18] # Filtering the data to get elements of same size

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [8]:

#Since the charachters are not ordinal , converting them into one Hot Encoding
import numpy as np
temp=np.array(OneHotEncoding(data['Molecule_sep'],key_value,data['length'].max()))
save(temp,np.array(key_value))
% time

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


In [9]:
# Reading the saved Nump file
import numpy as np
data_en=np.load('smiles.npz',allow_pickle=True)
key_value=data_en['key'].item()
data_en=data_en['ohe']

# LSTM Autoencoder
Reference : TowardsDataScience post on LSTM AutoEncoders by Chitta Ranjan


In [10]:
OneHotDecoding_helper(data_en[0],key_value)

'CC(C)(C)CCC(C)(C)C'

In [11]:
slicing_point=int(len(data_en)*0.75)
x_train=data_en[:slicing_point]
x_test=data_en[slicing_point:]

In [41]:

# The motive here is to create a Deep autoencoder
from keras import Sequential
from keras.layers import LSTM,Dense,RepeatVector
# define model
model = Sequential()
model.add(LSTM(18, activation='relu', input_shape=(18,18), return_sequences=True))
model.add(LSTM(8, activation='relu', return_sequences=False))
model.add(RepeatVector(18))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dense(18, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

model.fit(x_train,x_train,epochs = 15,
          batch_size=512,
          shuffle = True,
          validation_data = (x_test,x_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 18, 18)            2664      
                                                                 
 lstm_5 (LSTM)               (None, 8)                 864       
                                                                 
 repeat_vector_1 (RepeatVect  (None, 18, 8)            0         
 or)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 18, 64)            18688     
                                                                 
 lstm_7 (LSTM)               (None, 18, 128)           98816     
                                                                 
 dense_1 (Dense)             (None, 18, 18)            2322      
                                                      

In [42]:
# Seperating the encoder and the decoder
from keras.models import Model
encoder=Model(inputs=model.input,outputs=model.layers[2].output)
decoder=Model(inputs=model.layers[-3].input,outputs=model.output)

In [43]:
#Getting the encoded format of the data
encoded_data=encoder.predict(data_en)

encoded_data.shape

(112993, 18, 8)

In [44]:
gaussian=[] # Storing the outputs of individial neurons to different lists
for data in encoded_data:
  neuron_data=[]
  for layer in data: # 18 layers
    for value in layer : # Each layer has 2 values
      neuron_data.append(value)
  gaussian.append(neuron_data)
gaussian=np.array(gaussian)

In [45]:
hist=[[] for i in range(144)]
for data in gaussian:
  for i,value in enumerate(data):
    hist[i].append(value)

In [46]:
smiles=decoder.predict(encoded_data)

In [59]:
len(smiles)

112993

In [62]:
i=0
molecules=[]
for molecule in smiles[100000:]:
  smiles_string=OneHotDecoding_helper(molecule,key_value)
  if len(smilite.get_zincid_from_smile(smiles_string))>0:
    molecules.append(smiles_string)
    print(smiles_string)
  if i%50==0:
    print(i+1,' generations scanned')
  if len(molecules)>3:
    break
  i+=1

CC11CCCCCCC(C)(F)F
1  generations scanned
CC11CCCCCCCCCC(C)C
CC11CCCCCCCCCC(C)O
Invalid SMILE string CC1CCCC))CCC=C)C1N
CC11CCCCCCCCCC(C)N


In [64]:
print('The successful smiles string generated are ')
molecules

The successful smiles string generated are 


['CC11CCCCCCC(C)(F)F',
 'CC11CCCCCCCCCC(C)C',
 'CC11CCCCCCCCCC(C)O',
 'CC11CCCCCCCCCC(C)N']

# Checking if the molecules generated are present in the dataset or if they are new

In [78]:
for molecule in molecules:
  if len(data[data['Molecule']=='CC11CCCCCCCCCC(C)O'].index.values)==0:
    print(molecule,'  not found in dataset')
  else:
    print(molecule,' found in dataset')

CC11CCCCCCC(C)(F)F   not found in dataset
CC11CCCCCCCCCC(C)C   not found in dataset
CC11CCCCCCCCCC(C)O   not found in dataset
CC11CCCCCCCCCC(C)N   not found in dataset


In [79]:
# Getting zinc ID of the generated molecules
zinc_id=[]
for molecule in molecules:
  zinc_id.append(smilite.get_zincid_from_smile(molecule)) 

In [80]:
import pandas as pd
new_molecules=pd.DataFrame()
new_molecules['Generated Molecules']=molecules
new_molecules['Zinc_ID']=zinc_id
new_molecules

,Generated Molecules,Zinc_ID
0,CC11CCCCCCC(C)(F)F,[ZINC000117580970]
1,CC11CCCCCCCCCC(C)C,[ZINC000002560572]
2,CC11CCCCCCCCCC(C)O,"[ZINC000001700021, ZINC000002039357]"
3,CC11CCCCCCCCCC(C)N,"[ZINC000001687258, ZINC000095719379]"
